# Collier County demographics and turnout, by precinct

- This narrows the data down to age, race, and voter turnout for each office in Collier County's 2018 primary. 

- The code can be modified to focus on specific candidates or party affiliation of voters.

In [13]:
import pandas as pd

In [14]:
#importing precinct demographics

dm = pd.read_csv('precinct_demographics_collier.csv') 

#source (July): https://web.archive.org/web/20180904150351/https://www.colliervotes.com/Portals/Collier/Documents/pdf/MonthEndReports/2018%20JULY/July%20Summary%20Precinct%20Demographic%20Analysis.pdf
#most recent reports: https://www.colliervotes.com/Voting-System-Maps-Stats/Voter-Demographics
#converted to csv with https://tabula.technology/

In [15]:
dm.dropna(subset=['Total'], inplace=True) #dropping "total" for each category

In [16]:
dm.drop(columns="Unnamed: 8", inplace=True) #dropping blank column added by Tabula

In [17]:
dm.drop(columns=["DEM","REP","NPA","OTHER","LPF"], inplace=True) #dropping subcategories to reshape data

In [18]:
dm = dm.pivot(index="Precinct",columns="Unnamed: 1") #setting precinct as index

In [19]:
dm.to_csv('collier_precinct_dm.csv') #saving csv; fixed headers in Excel

In [20]:
dm = pd.read_csv('collier_precinct_dm.csv') #opening edited csv

In [21]:
results = pd.read_csv('PrecinctResults_2018-08-29T00_21_28.csv')
#source (2018 primary elections): https://web.archive.org/web/20180904151351/https://s3.amazonaws.com/results.voterfocus.com/enr/exports/reports/CLL/1920/PrecinctResults_2018-08-31T14:27:26.csv
#updates: https://enr.electionsfl.org/CLL/1920/Reports/

In [22]:
results = results[['PrecinctCode', 'TotalVotes','Contest']] #limiting to essential columns

In [23]:
dm = pd.merge(results, dm, how='inner', left_on='PrecinctCode', right_on='Precinct',
         sort=True) #merging results data with precinct data

In [24]:
turnout = pd.DataFrame(dm.groupby(['Contest','Precinct'])['TotalVotes'].sum()) 
#dataframe for turnout for each race, by precinct

In [25]:
dm = pd.merge(dm, turnout, on=['Contest','Precinct'])
#merging turnout numbers and the main dataframe

In [26]:
dm.drop_duplicates(['Contest','Precinct'],keep="first",inplace=True)
#dropping duplicates - only looking at overall turnout per race

In [27]:
dm.drop(['PrecinctCode','TotalVotes_x'], axis=1, inplace=True) #dropping repetitive columns

In [28]:
dm.rename(columns={"**TOTAL**": "Registered Voters", "TotalVotes_y": "Votes cast"}, inplace=True)
#renaming columns for clarity

In [29]:
turnout = dm[['Contest','Precinct','Votes cast']] #totalling turnout by office 

In [30]:
turnout = turnout.pivot(index="Precinct",columns="Contest") #pivoting dataframe and making Precinct the index

In [31]:
dm.drop_duplicates(['Precinct'], inplace=True, keep="first")
#dropping Precinct duplicates

In [32]:
dm = pd.merge(dm, turnout, on=['Precinct']) #merging main dataframe and turnout numbers

/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [33]:
all_turnout = pd.read_json('collier_turnout.json')
#source: voter turnout map at (used chrome dev tools to pull json from source code)
#http://web.archive.org/web/20180904184138/https://enr.electionsfl.org/CLL/1920/Map/VoterTurnout/
#updates: https://enr.electionsfl.org/CLL/1920/Map/VoterTurnout/

In [34]:
dm = pd.merge(dm, all_turnout, how='inner', left_on='Precinct', right_on='PrecinctCode',
         sort=True) 
#adding total turnout to original dataframe

In [35]:
dm.drop(['PrecinctCode','PrecinctId','PrecinctName','ReportingStatus','TotalVotes','Participating',\
         'Votes cast','Contest','Registered Voters'], axis=1,inplace=True)
#dropping unnecessary columns

In [36]:
dm = dm.replace({',':''},regex=True).apply(pd.to_numeric,1) #making numbers numeric

In [37]:
dm['Female'] = dm["BLACK FEMALE"] + dm["WHITE FEMALE"] + dm["HISPANIC FEMALE"]\
                    + dm["OTHER FEMALE"] #creating "Female" category

In [38]:
dm['Male']= dm["BLACK MALE"] + dm["WHITE MALE"] + dm["HISPANIC MALE"]\
                    + dm["OTHER MALE"] #creating "Male" category

In [39]:
dm = dm.set_index("Precinct")

In [40]:
pd.options.display.max_columns = None
dm.to_csv('collier_2018_primary_precincts.csv')